# Furniture Supply Store Transshipment Problem

Prerequisites

In [1]:
from pulp import *
import pulp as p

Step 1: Define input parameters using lists/dictionaries

In [2]:
# Nodes in the problem
Nodes = ['Node1', 'Node2', 'Node3', 'Node4', 'Node5', 'Node6']

# Supply and Demand values for each node
SupplyDemand = {
    'Node1': -30,  # Supply
    'Node2': -40,  # Supply
    'Node3': -20,  # Supply
    'Node4': 10,   # Demand
    'Node5': 30,   # Demand
    'Node6': 40    # Demand
}

# Define the arcs and corresponding shipping costs
Arcs = [
    ('Node1', 'Node3'), ('Node1', 'Node4'),
    ('Node2', 'Node4'), ('Node2', 'Node5'),
    ('Node3', 'Node4'), ('Node3', 'Node6'),
    ('Node4', 'Node6'), ('Node5', 'Node4'),
    ('Node5', 'Node6')
]

# Shipping costs on each arc
Cost = {
    ('Node1', 'Node3'): 20,
    ('Node1', 'Node4'): 30,
    ('Node2', 'Node4'): 10,
    ('Node2', 'Node5'): 20,
    ('Node3', 'Node4'): 20,
    ('Node3', 'Node6'): 40,
    ('Node4', 'Node6'): 30,
    ('Node5', 'Node4'): 50,
    ('Node5', 'Node6'): 30
}

Step 2: Define decision variables

In [3]:
Flows = p.LpVariable.dicts("X", Arcs, lowBound=0, upBound=None, cat=LpContinuous)

In [4]:
Flows

{('Node1', 'Node3'): X_('Node1',_'Node3'),
 ('Node1', 'Node4'): X_('Node1',_'Node4'),
 ('Node2', 'Node4'): X_('Node2',_'Node4'),
 ('Node2', 'Node5'): X_('Node2',_'Node5'),
 ('Node3', 'Node4'): X_('Node3',_'Node4'),
 ('Node3', 'Node6'): X_('Node3',_'Node6'),
 ('Node4', 'Node6'): X_('Node4',_'Node6'),
 ('Node5', 'Node4'): X_('Node5',_'Node4'),
 ('Node5', 'Node6'): X_('Node5',_'Node6')}

Step 3: Create a variable to hold the problem data

In [5]:
Transshipment = p.LpProblem("Furniture_Transshipment", p.LpMinimize)

Step 4: Define the objective function

In [6]:
Transshipment += lpSum(Flows[i] * Cost[i] for i in Arcs), "Total_Transportation_Cost"

In [7]:
Transshipment

Furniture_Transshipment:
MINIMIZE
20*X_('Node1',_'Node3') + 30*X_('Node1',_'Node4') + 10*X_('Node2',_'Node4') + 20*X_('Node2',_'Node5') + 20*X_('Node3',_'Node4') + 40*X_('Node3',_'Node6') + 30*X_('Node4',_'Node6') + 50*X_('Node5',_'Node4') + 30*X_('Node5',_'Node6') + 0
VARIABLES
X_('Node1',_'Node3') Continuous
X_('Node1',_'Node4') Continuous
X_('Node2',_'Node4') Continuous
X_('Node2',_'Node5') Continuous
X_('Node3',_'Node4') Continuous
X_('Node3',_'Node6') Continuous
X_('Node4',_'Node6') Continuous
X_('Node5',_'Node4') Continuous
X_('Node5',_'Node6') Continuous

Step 5: Define the constraints

In [8]:
for n in Nodes:
    if n in SupplyDemand:
        Transshipment += (
            lpSum(Flows[(o, d)] for (o, d) in Arcs if n == d) -
            lpSum(Flows[(o, d)] for (o, d) in Arcs if n == o)
            >= SupplyDemand[n], f"Flow_Balance_{n}"
        )

In [9]:
Transshipment

Furniture_Transshipment:
MINIMIZE
20*X_('Node1',_'Node3') + 30*X_('Node1',_'Node4') + 10*X_('Node2',_'Node4') + 20*X_('Node2',_'Node5') + 20*X_('Node3',_'Node4') + 40*X_('Node3',_'Node6') + 30*X_('Node4',_'Node6') + 50*X_('Node5',_'Node4') + 30*X_('Node5',_'Node6') + 0
SUBJECT TO
Flow_Balance_Node1: - X_('Node1',_'Node3') - X_('Node1',_'Node4') >= -30

Flow_Balance_Node2: - X_('Node2',_'Node4') - X_('Node2',_'Node5') >= -40

Flow_Balance_Node3: X_('Node1',_'Node3') - X_('Node3',_'Node4')
 - X_('Node3',_'Node6') >= -20

Flow_Balance_Node4: X_('Node1',_'Node4') + X_('Node2',_'Node4')
 + X_('Node3',_'Node4') - X_('Node4',_'Node6') + X_('Node5',_'Node4') >= 10

Flow_Balance_Node5: X_('Node2',_'Node5') - X_('Node5',_'Node4')
 - X_('Node5',_'Node6') >= 30

Flow_Balance_Node6: X_('Node3',_'Node6') + X_('Node4',_'Node6')
 + X_('Node5',_'Node6') >= 40

VARIABLES
X_('Node1',_'Node3') Continuous
X_('Node1',_'Node4') Continuous
X_('Node2',_'Node4') Continuous
X_('Node2',_'Node5') Continuous
X_('No

Step 6: Solve and Print Status

In [10]:
status = Transshipment.solve()
print("Status:", p.LpStatus[status])

Status: Optimal


Step 7: Print the value of the objective function

In [11]:
print("Total Transportation Cost = $", p.value(Transshipment.objective))

Total Transportation Cost = $ 2700.0


Step 8: Print the value of the decision variables

In [12]:
for var in Transshipment.variables():
    print(f"{var.name} = {var.varValue}")

X_('Node1',_'Node3') = 20.0
X_('Node1',_'Node4') = 0.0
X_('Node2',_'Node4') = 10.0
X_('Node2',_'Node5') = 30.0
X_('Node3',_'Node4') = 0.0
X_('Node3',_'Node6') = 40.0
X_('Node4',_'Node6') = 0.0
X_('Node5',_'Node4') = 0.0
X_('Node5',_'Node6') = 0.0
